In [1]:
"""
Created on Wed Jan 23 11:48:32 2019

@author: Andrew Welsh

This notebook is for doing model parameter tuning grid searches and feature selection using historical data, 
with intention of finding the optimal parameters to use for the live model fit. 

The data must be extracted via SQL prior to inclusion in this notebook (until a data connection directly to 
the database from Seth's machines is configured), then saved via CSV, which is read here. 

Intended use is for initial model development, as well as when new features are added.

Output: knowledge of optimal model parameters to attain best fit
"""

#import pypyodbc 
import pandas as pd
import churn_common as p #imports F2 score function, model_fit function

# imported data already has nulls cleaned and unused columns dropped
# churn_cloud_model_training_fit.csv is output from 'churn_cloud_model_training_fit.py' python script
cloud = pd.read_csv('churn_cloud_model_training_fit.csv', encoding='ISO-8859-1')

print('Query complete')
pd.options.display.max_columns = None
#cloud = cloud.iloc[1:5000, :].copy() #select first 5k rows for fast testing
cloud = cloud.loc[cloud['avg per line item invoiced in last 6 months'].notnull()] 


Query complete


In [2]:
#index column from CSV is imported as a separate column, labeled 'Unnamed: 0', therefore must be dropped before model fit
cloud.drop(['Unnamed: 0'], axis=1, inplace=True)

#generate TMK date column
cloud['time_month_key_dt']=pd.to_datetime(
        cloud['time_month_key'].apply(str).str.slice(stop=4)+"-"+
        cloud['time_month_key'].apply(str).str.slice(start=4)+"-01")

#hold out evaluation dataset of last 3 months
#Months are hard-coded here, use df['time_month_key'].max() to find max TMK, configure eval set to be 3 most recent months
#Example: max TMK 201812; eval data is 201810 to 201812 inclusive, cloud_num is 201809 and earlier
#min TMK is 201507
#max TMK is 201809

#define model training dataset, set to TMK < max(TMK)-3
cloud_train = cloud.loc[cloud['time_month_key']<201806]
cloud_train.set_index(['account_number','time_month_key'])

#hold out evaluation dataset of last 3 months
cloud_val = cloud.loc[(cloud['time_month_key']>=201806) & (cloud['time_month_key']<201809)]
cloud_val.set_index(['account_number','time_month_key'])

#the table head for the eval data should output below. Most accounts should have 3 rows each (3 latest TMKs).

account type - cloud uk  mi_mo - core  \
account_number time_month_key                                          
883600         201808                                0             1   
               201806                                0             1   
883643         201806                                0             1   
883642         201808                                0             1   
883699         201806                                0             0   
883643         201807                                0             1   
883689         201806                                0             1   
883661         201806                                0             1   
883701         201806                                0             0   
883719         201807                                0             0   
883731         201806                                0             1   
               201807                                0             1   
883723         201806                                0             1   
883732         201806                                0             0   
883743         201808                                0             1   
883732         201808                                0             0   
883752         201808                                0             1   
883754         201807                                0             1   
883777         201806                                0             0   
883811         201806                                0             1   
883777         201808                                0             0   
883921         201807                                0             1   
883769         201806                                0             1   
883811         201808                                0             1   
883918         201806                                0             0   
883810         201807                                0             0   
883918         201808                                0             0   
883933         201806                                0             1   
883841         201808                                0             0   
883955         201806                                0             1   
...                                                ...           ...   
10009315       201808                                1             1   
10009525       201807                                1             1   
10008289       201808                                1             0   
10008292       201808                                1             0   
10008482       201807                                1             1   
10008625       201808                                1             1   
10008007       201806                                1             1   
10006937       201806                                1             0   
10007229       201806                                1             1   
10007261       201808                                1             1   
10006883       201806                                1             1   
10005532       201808                                1             1   
10006140       201806                                1             1   
10006161       201807                                1             0   
10004739       201806                                1             1   
10005485       201808                                1             0   
10004382       201807                                1             1   
459832         201808                                0             1   
10003252       201807                                1             1   
10003425       201808                                1             1   
1000354        201807                                0             0   
10003614       201808                                1             1   
10001612       201808                                1             1   

In [3]:
#split datasets into low-med-hi revenue segments

#training dataset
cloud_train_low, cloud_train_mid, cloud_train_hi = p.split_df_revenue_segments(cloud_train, 'average_invoiced_last_12_months', low=1000, high=10000)

#validation dataset
cloud_val_low, cloud_val_mid, cloud_val_hi = p.split_df_revenue_segments(cloud_val, 'average_invoiced_last_12_months', low=1000, high=10000)

print("Train-test-eval split complete")

Train-test-eval split complete


In [6]:
#==================================================================================================================
#==================================================================================================================
# Resample minority class in training dataset
#==================================================================================================================
#==================================================================================================================

#for repeatable results, set seed equal to an integer; for random assignment, set seed to None
seed = 155

cloud_train_low_rus = p.AndrewsRandomUnderSampler(cloud_train_low, 'target', seed)
cloud_train_mid_rus = p.AndrewsRandomUnderSampler(cloud_train_mid, 'target', seed)
cloud_train_hi_rus = p.AndrewsRandomUnderSampler(cloud_train_hi, 'target', seed)

print('Minority class resampling complete')

Minority class resampling complete


In [5]:
#define lists of target (y) and predictor (x) columns
exclude_col = ['target'
               ,'average_invoiced_last_12_months'
               ,'account_number'
               ,'time_month_key'
               ,'month_order'
               ,'time_month_key_dt'
               ]

include_col = ['3 mth pct change number of device_status = computer no longer active'
               ]

#predictors = list(cloud_train.columns[~cloud_train.columns.isin(exclude_col)])
predictors = list(cloud_train.columns[cloud_train.columns.isin(include_col)])

#create copy of the X and Y columns
x1_train_res_low = cloud_train_low_rus[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_train_res_low = cloud_train_low_rus['target']

x1_train_res_mid = cloud_train_mid_rus[predictors]
y1_train_res_mid = cloud_train_mid_rus['target']

x1_train_res_hi = cloud_train_hi_rus[predictors]
y1_train_res_hi = cloud_train_hi_rus['target']

print("train ready")

#create copy of the X and Y columns for validation data
x_val_low = cloud_val_low[predictors] #uses list of columns excluding target (predictors) to select columns for x_low
y1_val_low = cloud_val_low['target']

x_val_mid = cloud_val_mid[predictors]
y1_val_mid = cloud_val_mid['target']

x_val_hi = cloud_val_hi[predictors]
y1_val_hi = cloud_val_hi['target']

print("eval ready")

train ready
eval ready


In [7]:
#Create time series split cross validators, cross-validated by time month key, on training dataset (test data generated 
#within the cross-validator routine, and used for grid search scoring in GridSearchCV API)
cvsplits = 4
cv_low = p.TimeSeriesSplitTMK(n_splits=cvsplits).split(cloud_train_low_rus)
cv_mid = p.TimeSeriesSplitTMK(n_splits=cvsplits).split(cloud_train_mid_rus)
cv_hi = p.TimeSeriesSplitTMK(n_splits=cvsplits).split(cloud_train_hi_rus)

In [8]:
#import libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import make_scorer, fbeta_score, matthews_corrcoef
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import time
import datetime
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [ ]:
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#Initial model run with no parameters tuned
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================

xgb_final_low = XGBClassifier(
        learning_rate=0.05, 
        n_estimators=1100, 
        max_depth=13, 
        max_features=13, 
        subsample=0.7,
        min_child_weight=1,
        gamma=0, 
        colsample_bytree=0.7, 
        scale_pos_weight=1, 
        reg_alpha=0.1,
        reg_lambda=1,
        random_state=seed, 
        eval_metric='auc',
        objective='binary:logistic',
        nthread=128
        )

xgb_final_mid = XGBClassifier(
        learning_rate=0.05, 
        n_estimators=1300, 
        max_depth=9, 
        max_features=13, 
        subsample=0.7,
        min_child_weight=1,
        gamma=0, 
        colsample_bytree=0.7, 
        scale_pos_weight=1, 
        reg_alpha=1e-05,
        reg_lambda=0.01,
        random_state=seed, 
        eval_metric='auc',
        objective='binary:logistic',
        nthread=128
        )

xgb_final_hi = XGBClassifier(
        learning_rate=0.05, 
        n_estimators=1300, 
        max_depth=9, 
        max_features=13, 
        subsample=0.7,
        min_child_weight=1,
        gamma=0, 
        colsample_bytree=0.7, 
        scale_pos_weight=1, 
        reg_alpha=1e-05,
        reg_lambda=1,
        random_state=seed, 
        eval_metric='auc',
        objective='binary:logistic',
        nthread=128
        )

p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), x_val_low, y1_val_low, predictors)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), x_val_mid, y1_val_mid, predictors)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(), x_val_hi, y1_val_hi, predictors)

In [9]:
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#PARAMETER TUNING
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================

#defining the grid search metric scorer. Using F2 scores; this weighs the recall twice as much as the precision
f2_scorer = make_scorer(fbeta_score, beta=2)

#Using F0.5 scores weighs the precision twice as much as the recall
#f05_scorer = make_scorer(fbeta_score, beta=0.5)

#mcc = make_scorer(matthews_corrcoef)

In [10]:
print('Run first grid search for parameter tuning for low revenue accts')

#
#
#
#Tune learning rate, gamma and number of estimators
#
#
#

param_test1 = {
  'learning_rate':[0.005,0.01],#test values of 0.05 to 0.3. must be between 0 and 1
#  'gamma':[i/2 for i in range(0,2)], #test values of 0 to 3, in steps of 0.5. must be non-negative, default 0
#  'gamma':[0,5,10], #test values of 1, 5 and 10. default is 0, but overfitting is a problem
  'n_estimators':range(500,1501,500) #test values of 500 to 1500, in steps of 500
}

start_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

g1_low = GridSearchCV(estimator = XGBClassifier(
#        learning_rate=0.1, 
#        n_estimators=500, 
        max_depth=13, 
        max_features=13, 
        subsample=0.8,
        min_child_weight=1,
        gamma=0, 
        colsample_bytree=0.8, 
        scale_pos_weight=1, 
        reg_alpha=0.1,
        reg_lambda=1e-05,
        random_state=seed, 
        eval_metric='auc',
        objective='binary:logistic', #try binary:hinge?
        max_delta_step=0,
        nthread=-1), param_grid = param_test1, scoring=f2_scorer, n_jobs=-1,iid=False, cv=cv_low
                     )
#1st iteration {'gamma': 1, 'learning_rate': 0.01, 'n_estimators': 1250}

g1_mid = GridSearchCV(estimator = XGBClassifier(
#        learning_rate=0.1, 
#        n_estimators=500, 
        max_depth=13, 
        max_features=13, 
        subsample=0.8,
        min_child_weight=1,
        gamma=0, 
        colsample_bytree=0.8, 
        scale_pos_weight=1, 
        reg_alpha=0.1,
        reg_lambda=1e-05,
        random_state=seed, 
        eval_metric='auc',
        objective='binary:logistic',
        max_delta_step=0,
        nthread=-1), param_grid = param_test1, scoring=f2_scorer, n_jobs=-1,iid=False, cv=cv_mid
                     )
#1st iteration {'gamma': 1, 'learning_rate': 0.005, 'n_estimators': 1000}

g1_hi = GridSearchCV(estimator = XGBClassifier(
#        learning_rate=0.1, 
#        n_estimators=1500, 
        max_depth=13, 
        max_features=13, 
        subsample=0.8,
        min_child_weight=1,
        gamma=0, 
        colsample_bytree=0.7, 
        scale_pos_weight=1, 
        reg_alpha=0.1,
        reg_lambda=1e-05,
        random_state=seed, 
        eval_metric='auc',
        objective='binary:logistic',
        max_delta_step=0,
        nthread=-1), param_grid = param_test1, scoring=f2_scorer, n_jobs=-1,iid=False, cv=cv_hi
                     )
#1st iteration {'gamma': 0, 'learning_rate': 0.005, 'n_estimators': 1500 (tested 1000-1500)}
#2nd iteration {'gamma': 0, 'learning_rate': 0.01, 'n_estimators': 1500 (tested 1500-2000)}

g1_low.fit(x1_train_res_low,y1_train_res_low)
g1_mid.fit(x1_train_res_mid,y1_train_res_mid)
g1_hi.fit(x1_train_res_hi,y1_train_res_hi)


end_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

Run first grid search for parameter tuning for low revenue accts


(low revenue)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eval_metric='auc', gamma=0.0,
       learning_rate=0.1, max_delta_step=0, max_depth=13, max_features=13,
       min_child_weight=1, missing=None, n_estimators=1400, n_jobs=1,
       nthread=-1, objective='binary:logistic', random_state=15,
       reg_alpha=0.1, reg_lambda=1e-05, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [11]:
g1_low_params = g1_low.best_estimator_
g1_mid_params = g1_mid.best_estimator_
g1_hi_params = g1_hi.best_estimator_

print(g1_low.best_estimator_)
print(g1_low.best_params_)
print(g1_low.best_score_)

print(g1_mid.best_estimator_)
print(g1_mid.best_params_)
print(g1_mid.best_score_)

print(g1_hi.best_estimator_)
print(g1_hi.best_params_)
print(g1_hi.best_score_)

print(start_time)
print(end_time)

print(g1_low.cv_results_)
print(g1_mid.cv_results_)
print(g1_hi.cv_results_)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=13, max_features=13,
       min_child_weight=1, missing=None, n_estimators=1500, n_jobs=1,
       nthread=-1, objective='binary:logistic', random_state=155,
       reg_alpha=0.1, reg_lambda=1e-05, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)
{'learning_rate': 0.01, 'n_estimators': 1500}
0.8499199685075913
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0,
       learning_rate=0.005, max_delta_step=0, max_depth=13,
       max_features=13, min_child_weight=1, missing=None,
       n_estimators=1000, n_jobs=1, nthread=-1,
       objective='binary:logistic', random_state=155, reg_alpha=0.1,
       reg_lambda=1e-05, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)
{'learning_rate': 0.005, 'n_estimators'

In [ ]:
Low:
    XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eval_metric='auc', gamma=0,
       learning_rate=0.005, max_delta_step=0, max_depth=13,
       max_features=13, min_child_weight=1, missing=None,
       n_estimators=1750, n_jobs=1, nthread=-1,
       objective='binary:logistic', random_state=17, reg_alpha=0.1,
       reg_lambda=1e-05, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)
{'learning_rate': 0.005, 'n_estimators': 1750}
0.7607220178419795
Mid: 
    XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, eval_metric='auc', gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=13, max_features=13,
       min_child_weight=1, missing=None, n_estimators=1500, n_jobs=1,
       nthread=-1, objective='binary:logistic', random_state=17,
       reg_alpha=0.1, reg_lambda=1e-05, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)
{'learning_rate': 0.01, 'n_estimators': 1500}
0.7877350749486776

In [ ]:
#
#
#
#
#Tune max_delta_step, max_depth and min_child_weight
#
#XGBoost docs say "Maximum delta step we allow each leaf output to be. If the value is set 
#to 0, it means there is no constraint. If it is set to a positive value, it can help making 
#the update step more conservative. Usually this parameter is not needed, but it might help 
#in logistic regression when class is extremely imbalanced. Set it to value of 1-10 might 
#help control the update." 
#
#max_delta_step range is 0 to infinity
#
#
#
#

param_test2 = {
 'max_depth':range(11,16,2)
 ,'min_child_weight':range(1,3,1)
 ,'max_delta_step':range(0,3,1) #try values 0 through 2, increment by 1
}

#Create time series split cross validators, cross-validated by time month key, on training dataset (test data generated 
#within the cross-validator routine, and used for grid search scoring in GridSearchCV API)
cvsplits = 3
cv_low = p.TimeSeriesSplitTMK(n_splits=cvsplits).split(cloud_train_low_rus)
cv_mid = p.TimeSeriesSplitTMK(n_splits=cvsplits).split(cloud_train_mid_rus)
cv_hi = p.TimeSeriesSplitTMK(n_splits=cvsplits).split(cloud_train_hi_rus)

start_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

#g2_low = GridSearchCV(estimator = XGBClassifier(g1_low_params), param_grid = param_test2, scoring=f2_scorer
#                      ,n_jobs=-1,iid=False, cv=3)


#g2_mid = GridSearchCV(estimator = XGBClassifier(g1_mid_params), param_grid = param_test2, scoring=f2_scorer
#                      ,n_jobs=-1,iid=False, cv=3)

#g2_hi = GridSearchCV(estimator = XGBClassifier(g1_hi_params), param_grid = param_test2, scoring=f2_scorer
#                     ,n_jobs=-1,iid=False, cv=3)

g2_low = GridSearchCV(estimator = XGBClassifier(
#{'learning_rate': 0.005, 'n_estimators': 1750}
        base_score=0.5,
        booster='gbtree',
        colsample_bylevel=1,
        colsample_bytree=0.7,
        gamma=10, 
        learning_rate=0.0001,
#        max_delta_step=1, 
#        max_depth=15, 
#        min_child_weight=1, 
        missing=None,
        n_estimators=1500, 
        random_state=seed,
        reg_alpha=0.1,
        reg_lambda=1e-05,
        scale_pos_weight=1,
        silent=True,
        subsample=0.8,
        eval_metric='auc',
        objective='binary:logistic',
        nthread=-1), param_grid = param_test2, scoring=f2_scorer
                      ,n_jobs=-1,iid=False, cv=cv_low)

g2_mid = GridSearchCV(estimator = XGBClassifier(
        base_score=0.5,
        booster='gbtree',
        colsample_bylevel=1,
        colsample_bytree=0.9,
        gamma=1, 
        learning_rate=0.01,
#        max_delta_step=1, 
#        max_depth=15, 
#        min_child_weight=1, 
        missing=None,
        n_estimators=1500, 
        random_state=seed,
        reg_alpha=0.01,
        reg_lambda=0.1,
        scale_pos_weight=1,
        silent=True,
        subsample=0.7,
        eval_metric='auc',
        objective='binary:logistic',
        nthread=-1), param_grid = param_test2, scoring=f2_scorer
                      ,n_jobs=-1,iid=False, cv=cv_mid)
#1st iteration: {'max_delta_step': 1, 'max_depth': 11, 'min_child_weight': 1}

g2_hi = GridSearchCV(estimator = XGBClassifier(
        learning_rate=0.01, 
        n_estimators=1500, 
        max_depth=13, 
        max_features=13, 
        subsample=0.8,
        min_child_weight=1,
        gamma=0, 
        colsample_bytree=0.7, 
        scale_pos_weight=1, 
        reg_alpha=0.1,
        reg_lambda=1e-05,
        random_state=seed, 
        eval_metric='auc',
        objective='binary:logistic',
        max_delta_step=0,
        nthread=-1), param_grid = param_test2, scoring=f2_scorer, n_jobs=-1,iid=False, cv=cv_hi)
#1st iteration {'max_delta_step': 1, 'max_depth': 11, 'min_child_weight': 1}

g2_low.fit(x1_train_res_low,y1_train_res_low)
g2_mid.fit(x1_train_res_mid,y1_train_res_mid)
g2_hi.fit(x1_train_res_hi,y1_train_res_hi)

end_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
#g2_low_params = g2_low.best_estimator_
#g2_mid_params = g2_mid.best_estimator_
g2_hi_params = g2_hi.best_estimator_

#print(g2_low.best_estimator_)
#print(g2_low.best_params_)
#print(g2_low.best_score_)

#print(g2_mid.best_estimator_)
#print(g2_mid.best_params_)
#print(g2_mid.best_score_)

print(g2_hi.best_estimator_)
print(g2_hi.best_params_)
print(g2_hi.best_score_)

print(start_time)
print(end_time)

#print(g2_low.cv_results_)
#print(g2_mid.cv_results_)
print(g2_hi.cv_results_)


In [ ]:
#
#
#
#
#Tune Subsample and colsample_bytree
#
#
#
#
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)], #test values of 0.0 to 0.4, in steps of 0.1
 'subsample':[i/10.0 for i in range(7,11)], #test values of 0.7 to 1.0, in steps of 0.1
 'colsample_bytree':[0.7,0.8,0.9,1]
}

start_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

#g3_low = GridSearchCV(estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=1,
#       max_depth=19, min_child_weight=1, missing=None, n_estimators=2400,
#       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
#       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#       silent=True, subsample=1), param_grid = param_test3, scoring=f2_scorer
#                      ,n_jobs=-1,iid=False, cv=3)


g3_mid = GridSearchCV(estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=1,
       max_depth=15, min_child_weight=1, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), param_grid = param_test3, scoring=f2_scorer
                      ,n_jobs=-1,iid=False, cv=2)


g3_hi = GridSearchCV(estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=21, min_child_weight=1, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1), param_grid = param_test3, scoring=f2_scorer
                     ,n_jobs=-1,iid=False, cv=2)

#g3_low.fit(x1_train_res_low,y1_train_res_low)
g3_mid.fit(x1_train_res_mid,y1_train_res_mid)
g3_hi.fit(x1_train_res_hi,y1_train_res_hi)

end_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
#g3_low_params = g3_low.best_estimator_
g3_mid_params = g3_mid.best_estimator_
g3_hi_params = g3_hi.best_estimator_

#print(g3_low.cv_results_)
#print(g3_low.best_estimator_)
#print(g3_low.best_params_)
#print(g3_low.best_score_)

#print(g3_mid.cv_results_)
print(g3_mid.best_estimator_)
print(g3_mid.best_params_)
print(g3_mid.best_score_)

#print(g3_hi.cv_results_)
print(g3_hi.best_estimator_)
print(g3_hi.best_params_)
print(g3_hi.best_score_)

print(start_time)
print(end_time)

In [ ]:
#
#
#
#
#Tune Regularization alpha (L1)
# and lambda (L2)
#
#
#
#
param_test4 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
 'reg_lambda':[1e-5, 1e-2, 0.1, 1, 10, 50, 100]
}

start_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

#g4_low = GridSearchCV(estimator = XGBClassifier(g3_low_params), param_grid = param_test4, scoring=f2_scorer
#                      ,n_jobs=-1,iid=False, cv=2)


g4_mid = GridSearchCV(estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.0, learning_rate=0.1,
       max_delta_step=1, max_depth=15, min_child_weight=1, missing=None,
       n_estimators=2000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.7), param_grid = param_test4, scoring=f2_scorer
                      ,n_jobs=-1,iid=False, cv=2)


g4_hi = GridSearchCV(estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.0, learning_rate=0.1,
       max_delta_step=0, max_depth=21, min_child_weight=1, missing=None,
       n_estimators=2000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.7), param_grid = param_test4, scoring=f2_scorer
                     ,n_jobs=-1,iid=False, cv=2)

#g4_low.fit(x1_train_res_low,y1_train_res_low)
g4_mid.fit(x1_train_res_mid,y1_train_res_mid)
g4_hi.fit(x1_train_res_hi,y1_train_res_hi)

end_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
#g4_low_params = g4_low.best_estimator_
g4_mid_params = g4_mid.best_estimator_
g4_hi_params = g4_hi.best_estimator_

#print(g4_low.cv_results_)
#print(g4_low.best_estimator_)
#print(g4_low.best_params_)
#print(g4_low.best_score_)

#print(g4_mid.cv_results_)
print(g4_mid.best_estimator_)
print(g4_mid.best_params_)
print(g4_mid.best_score_)

#print(g4_hi.cv_results_)
print(g4_hi.best_estimator_)
print(g4_hi.best_params_)
print(g4_hi.best_score_)

print(start_time)
print(end_time)

In [ ]:
#
#
#
#
#Tune learning rate and n_estimators
#
#
#
#
param_test5 = {
 'max_delta_step': range(0,6,1)
}

start_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

g5_low = GridSearchCV(estimator = XGBClassifier(g4_low_params), param_grid = param_test5, scoring=f2_scorer
                      ,n_jobs=-1,iid=False, cv=2)


g5_mid = GridSearchCV(estimator = XGBClassifier(g4_mid_params), param_grid = param_test5, scoring=f2_scorer
                      ,n_jobs=-1,iid=False, cv=2)


g5_hi = GridSearchCV(estimator = XGBClassifier(g4_hi_params), param_grid = param_test5, scoring=f2_scorer
                     ,n_jobs=-1,iid=False, cv=2)

g5_low.fit(x1_train_res_low,y1_train_res_low)
g5_mid.fit(x1_train_res_mid,y1_train_res_mid)
g5_hi.fit(x1_train_res_hi,y1_train_res_hi)

end_time = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
g5_low_params = g5_low.best_estimator_
g5_mid_params = g5_mid.best_estimator_
g5_hi_params = g5_hi.best_estimator_

#print(g5_low.cv_results_)
print(g5_low.best_estimator_)
print(g5_low.best_params_)
print(g5_low.best_score_)

#print(g5_mid.cv_results_)
print(g5_mid.best_estimator_)
print(g5_mid.best_params_)
print(g5_mid.best_score_)

#print(g5_hi.cv_results_)
print(g5_hi.best_estimator_)
print(g5_hi.best_params_)
print(g5_hi.best_score_)

print(start_time)
print(end_time)

In [ ]:
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================
#Final Model
#==================================================================================================================
#==================================================================================================================
#==================================================================================================================

xgb_final_low = XGBClassifier(
        colsample_bylevel=1,
        colsample_bytree=0.7,
        gamma=0, 
        learning_rate=0.005,
        max_delta_step=2, 
        max_depth=14, 
        max_features=13, 
        min_child_weight=0, 
        n_estimators=1750, 
#        random_state=seed,
        reg_alpha=0.1,
        reg_lambda=1e-05,
        scale_pos_weight=1,
        silent=True,
        subsample=0.8,
        eval_metric='auc',
        objective='binary:logistic',
        nthread=-1)

xgb_final_mid = XGBClassifier(
        colsample_bylevel=1,
        colsample_bytree=0.9,
        gamma=1, 
        learning_rate=0.01,
        max_delta_step=1, 
        max_depth=11, 
        min_child_weight=1, 
        missing=None,
        n_estimators=1500, 
#        random_state=seed,
        reg_alpha=0.01,
        reg_lambda=0.1,
        scale_pos_weight=1,
        silent=True,
        subsample=0.7,
        eval_metric='auc',
        objective='binary:logistic',
        nthread=-1)


xgb_final_hi = XGBClassifier(
        colsample_bylevel=1,
        colsample_bytree=0.7,
        gamma=1, 
        learning_rate=0.01,
        max_delta_step=0, 
        max_depth=13, 
        max_features=13, 
        min_child_weight=1, 
        missing=None,
        n_estimators=1500, 
#        random_state=seed,
        reg_alpha=0.01,
        reg_lambda=1e-05,
        scale_pos_weight=1,
        silent=True,
        subsample=0.8,
        eval_metric='auc',
        objective='binary:logistic',
        nthread=-1)

p.modeleval(xgb_final_low, x1_train_res_low, y1_train_res_low.values.ravel(), x_val_low, y1_val_low, predictors)
p.modeleval(xgb_final_mid, x1_train_res_mid, y1_train_res_mid.values.ravel(), x_val_mid, y1_val_mid, predictors)
p.modeleval(xgb_final_hi, x1_train_res_hi, y1_train_res_hi.values.ravel(), x_val_hi, y1_val_hi, predictors)

In [ ]:
print('low revenue features')
fi_final_low = pd.Series(xgb_final_low.get_booster().get_fscore()).sort_values(ascending=False)
print(fi_final_low.head())
print('mid revenue features')
fi_final_mid = pd.Series(xgb_final_mid.get_booster().get_fscore()).sort_values(ascending=False)
print(fi_final_mid.head())
print('hi revenue features')
fi_final_hi = pd.Series(xgb_final_hi.get_booster().get_fscore()).sort_values(ascending=False)
print(fi_final_hi.head())